Basics
======

Atooms provides a high-level interface to the main objects of
particle-based simulations. It mostly focuses on classical molecular
dynamics and Monte Carlo simulations, but it is not limited to that. For
instance, it can be used to model lattice models such as TASEP or
kinetically constrained models.

We will start by having a look at the basic objects of particle-based
simulations and how to store them on a file.

Particles\' properties
----------------------

Particles\' positions are stored as numpy arrays, but we can pass a
simple list with x, y, z coordinates when we create them


In [ ]:
  from atooms.system.particle import Particle
  particle = Particle(position=[1.0, 0.0, 0.0])
  print(particle.position, type(particle.position))

Particles can live in an arbitrary number of spatial dimensions


In [ ]:
  particle = Particle(position=[1.0, 0.0, 0.0, 0.0, 0.0])
  print(len(particle.position))

By default, they also have a few more properties such as velocity,
chemical species, mass and radius. They can all be altered at will or
even set to None.


In [ ]:
  import numpy
  particle = Particle(position=[1.0, 0.0, 0.0], velocity=[1.0, 0.0, 0.0])
  particle.species = 'Na'
  particle.position += numpy.array([0.0, 1.0, 1.0])
  particle.velocity *= 2
  particle.radius = None  # point particles have no radius
  print(particle)

You may want to add physical properties to particles, like charge or
whatever. Of course, in python you can do it very easily


In [ ]:
  particle.charge = -1.0

This won\'t break anything!

Dealing with velocities
-----------------------

You may not need velocities at all (for instance because you are working
with Monte Carlo simulations), but if you do, atooms provides a few
useful methods and functions. For instance, you can assign velocity from
a Maxwell-Boltzmann distribution at a temperature T.


In [ ]:
  particle = [Particle() for i in range(1000)]
  for p in particle:
      p.maxwellian(T=1.0)
  ekin = sum([p.kinetic_energy for p in particle])
  ndim = 3
  ndof = len(particle) * ndim
  T = 2.0 / ndof * ekin
  print(T)

Doing so will leave a non-zero total momentum, but we can fix it (note
that all masses are equal)


In [ ]:
  from atooms.system.particle import fix_total_momentum, cm_velocity
  print(cm_velocity(particle))
  fix_total_momentum(particle)
  print(cm_velocity(particle))

Simulation cell
---------------

To avoid major finite size effects, we enclose particles in a cell with
periodic boundary conditions. By convention, the cell origin is in the
origin of the reference frame.


In [ ]:
  from atooms.system.cell import Cell
  L = 2.0
  cell = Cell(side=[L, L, L])
  print(cell.side, cell.volume)

Atooms provides means to fold particles back in the \"central\"
simulation cell, i.e. the one centered at the origin at the reference
frame. For simplicity, let us work with particles in 1d.


In [ ]:
  cell = Cell(side=1.0)
  particle = Particle(position=2.0)  # particle outside the central cell
  particle.fold(cell)
  print(particle.position)

The particle is now folded back at the origin.

A related method returns the nearest periodic image of a given particle
with respect to another particle


In [ ]:
  particle_1 = Particle(position=-0.45)
  particle_2 = Particle(position=+0.45)
  image = particle_1.nearest_image(particle_2, cell, copy=True)
  print(image)

The system object
-----------------

Objects like particles and the simulation cell can be gathered in an
instance of a god-like class called System. The system contains all the
relevant physical objects of your simulation. Reservoirs like
thermostats, barostats and particle reservoirs can be added as well.
These objects are placeholders for thermodynamic state variables like
temperature, pressure or chemical potential. Any class meant to describe
the interaction between particles also belongs to the system.

Let us build a system with a few particles in a cell and use the system
methods to modify the system density and temperature. Note that density
and temperature are python properties and thus modify the attributes of
particles and cell under the hoods.


In [ ]:
  from atooms.system import System
  system = System(particle=[Particle() for i in range(100)],
		  cell=Cell([10.0, 10.0, 10.0]))
  system.density = 1.2  # equivalent to system.set_density(1.2)
  system.temperature = 1.5  # equivalent to system.set_temperature(1.2)
  print(system.density, system.temperature)

Note that the system temperature is the kinetic one and need not
coincide with the one of the thermostat.


In [ ]:
  from atooms.system import Thermostat
  system.thermostat = Thermostat(temperature=1.0)
  system.temperature = 1.5  # equivalent to system.set_temperature(1.2)
  print(system.temperature, system.thermostat.temperature)

Read and write trajectory files
-------------------------------

To write the state of the system to a file, we use a Trajectory class.
Trajectories are composed of multiple frames, each one holding the state
of the system at a given step during the simulation. We use a basic xyz
format and read the file back to see how it looks like.


In [ ]:
  from atooms.trajectory import TrajectoryXYZ

  system = System(particle=[Particle() for i in range(4)],
		  cell=Cell([10.0, 10.0, 10.0]))

  with TrajectoryXYZ('test.xyz', 'w') as th:
    th.write(system, step=0)

  # Read the xyz file back as plain text
  with open('test.xyz') as fh:
    print fh.read()

We can customize the output of the xyz trajectory by modifying the list
of particle fields to be written.


In [ ]:
  for p in system.particle:
    p.charge = -1.0

  with TrajectoryXYZ('test.xyz', 'w', fields=['position', 'charge']) as th:
    th.write(system, step=0)

  with open('test.xyz') as fh:
    print fh.read()

Of course, we can write multiple frames by calling write() repeatedly.


In [ ]:
  with TrajectoryXYZ('test.xyz', 'w') as th:
    for i in range(3):
      th.write(system, step=i*10)

To get the system back we read the trajectory. Trajectories support
iteration and indexing, just like lists.


In [ ]:
  with TrajectoryXYZ('test.xyz') as th:
    # First frame
    system = th[0]
    print(system.particle[0].position, system.cell.side)

    # Last frame
    system = th[-1]
    print(system.particle[0].position, system.cell.side)
  
    # Iterate over all frames
    for i, system in enumerate(th):
      print(th.steps[i], system.particle[0].position)

A minimal simulation backend
============================

Within atooms, **simulations** are high-level classes that encapsulate
some common tasks and provide a consistent interface to the user, while
**backends** are classes that actually make the system evolve. Here we
implement a minimal backend to run a simulation.

At a very minimum, a backend is a class that provides

-   a **system** instance variable, which should (mostly) behave like
    atooms.system.System.
-   a **run()** method, which evolves the system for a prescribed number
    of steps (passed as argument)

Optionally, the backend may hold a reference to a trajectory class,
which can be used to checkpoint the simulation or to write
configurations to a file. This is however not required in a first stage.
*Note: before atooms 1.5.0, backends also had to implement a
write~checkpoint~() method and they were required to hold a reference to
Trajectory. Since 1.5.0 this is no longer necessary.*

We set up a bare-bones simulation backend building on the native System
class


In [ ]:
  from atooms.system import System
  
  class BareBonesBackend(object):
      
      def __init__(self):
          self.system = System()
  
      def run(self, steps):
          for i in range(steps):
              pass
  
  # The backend is created and wrapped by a simulation object.
  # Here we first call the run() method then run_until()
  from atooms.simulation import Simulation
  backend = BareBonesBackend()
  simulation = Simulation(backend)
  simulation.run(10)
  simulation.run_until(30)
  assert simulation.current_step == 30
  
  # This time we call run() multiple times 
  simulation = Simulation(backend)
  simulation.run(10)
  simulation.run(20)
  assert simulation.current_step == 30  
  
  # Increase verbosity to see a meaningful log
  from atooms.core.utils import setup_logging
  setup_logging(level=20)
  simulation = Simulation(backend)
  simulation.run(10)  

``` {.example}
# 
# atooms simulation via <__main__.BareBonesBackend object at 0x7f2091065a50>
# 
# version: 1.5.0+1.5.0-4-g8f32a9 (2018-09-07)
# atooms version: 1.5.0+1.5.0-4-g8f32a9 (2018-09-07)
# simulation started on: 2018-09-07 at 10:59
# output path: None
# backend: <__main__.BareBonesBackend object at 0x7f2091065a50>
# 
# target target_steps: 10
# 
# 
# starting at step: 0
# simulation ended successfully: reached target steps 10
# 
# final steps: 10
# final rmsd: 0.00
# wall time [s]: 0.00
# average TSP [s/step/particle]: nan
# simulation ended on: 2018-09-07 at 10:59
```

A simple random walk
====================

We implement a simple random walk in 3d. This requires adding code to
the backend run() method to actually move the particles around.

We start by building an empty system. Then we add a few particles and
place them at random in a cube. Finally, we write a backend that
displaces each particle randomly over a cube of prescribed side.


In [ ]:
  import numpy
  from atooms.system import System

  # There are no particles at the beginning
  system = System()
  assert len(system.particle) == 0

  # Add particles
  from atooms.system.particle import Particle
  from random import random
  L = 10
  for i in range(1000):
      p = Particle(position=[L * random(), L * random(), L * random()])
      system.particle.append(p)

  class RandomWalk(object):

      def __init__(self, system, delta=1.0):
	  self.system = system
	  self.delta = delta

      def run(self, steps):
	  for i in range(steps):
	      for p in self.system.particle:
		  dr = numpy.array([random()-0.5, random()-0.5, random()-0.5])
		  dr *= self.delta
		  p.position += dr


The Simulation class provides a callback mechanism to allow execution of
arbitrary code during the simulation. This can be used to write logs or
particle configurations to file, or to perform on-the-fly calculations
of the system properties. Callbacks are plain function that accept the
simulation object as first argument. They are called at prescribed
intervals during the simulation.

Here we measure the mean square displacement (MSD) of the particles to
make sure that the system displays a regular diffusive behavior
$MSD \sim t$


In [ ]:
  from atooms.simulation import Simulation
  simulation = Simulation(RandomWalk(system))

  # We add a callback that computes the MSD every 10 steps
  # We store the result in a dictionary passed to the callback
  msd_db = {}
  def cbk(sim, initial_position, db):
      msd = 0.0
      for i, p in enumerate(sim.system.particle):
	  dr = p.position - initial_position[i]
	  msd += numpy.sum(dr**2)
      msd /= len(sim.system.particle)
      db[sim.current_step] = msd

  # We will execute the callback every 10 steps
  simulation.add(cbk, 10, initial_position=[p.position.copy() for p in
					    system.particle], db=msd_db)
  simulation.run(50)

  # The MSD should increase linearly with time
  time = sorted(msd_db.keys())
  msd = [msd_db[t] for t in time]

  print time, msd
  import matplotlib.pyplot as plt
  plt.cla()
  plt.plot(time, msd, '-o')
  plt.xlabel("t")
  plt.ylabel("MSD")
  plt.savefig('msd.png')

Here is the MSD as a function of time. It should look linear.
![](../msd.png)

Particles on a lattice
======================

We want to simulate a system where particles can only be located at
discrete sites, say a one-dimensional lattice or perhaps a network with
a complex topology. Particle positions can be described as simple
integers, holding the index of the site on which a particle is located.
We create such a system and then write it to a file in xyz format


In [ ]:
  import numpy
  from atooms.system import System, Particle
  
  # Build model system with integer coordinates
  particle = [Particle() for i in range(3)]
  particle[0].position = 0
  particle[1].position = 1
  particle[2].position = 2
  system = System(particle=particle)
  
  # Write xyz trajectory
  from atooms.trajectory import TrajectoryXYZ
  with TrajectoryXYZ('test.xyz', 'w') as th:
    th.write(system, 0)
  
  # Read the xyz file back as plain text
  with open('test.xyz') as fh:
    print(fh.read())

Everything went fine. However, we have to tweak things a bit when
reading the particles back, to avoid positions being transformed to
arrays of floats instead of integers. This can be done with the help of
a callback that transforms the system accordingly as we read the
trajectory.


In [ ]:
  # Read file as an xyz trajectory 
  with TrajectoryXYZ('test.xyz') as th:

    # We add a callback to read positions as simple integers
    # Otherwise they are read as numpy arrays of floats.
    def modify(system):      
      for p in system.particle:
	p.position = int(p.position[0])
	p.velocity = None
	p.radius = None
      return system
    th.add_callback(modify)

    for p in th[0].particle:
      print p